In [1]:
import os
from dotenv import load_dotenv
from bs4 import BeautifulSoup
import requests
import pandas as pd
import requests

In [2]:
load_dotenv()

True

In [3]:
def get_soup(url):
    page = requests.get(url,headers = {"User-Agent":"Mozilla/5.0","origin":"https://www.premierleague.com"})
    soup = BeautifulSoup(page.content, 'html.parser')
    return soup

#Testing with Bunrley Vs Arsenal

In [4]:
match_soup = get_soup(f'https://www.premierleague.com/match/38679')
timeline_div = match_soup.find('div',{'class':'timeLineEventsContainer'})

match_df = pd.DataFrame([],columns=['team','player','event','minute'])
matchweek = match_soup.find('div',{'class':'long'}).text
match_date = match_soup.find('div',{'class':'matchDate'})['data-kickoff']
referee = match_soup.find('div',{'class':'referee'}).text
stadium = match_soup.find('div',{'class':'stadium'}).text
attendance = match_soup.find('div',{'class':'attendance'}).text
kick_off = match_soup.find('strong',{'class':'renderKOContainer'}).text
match_details_df = pd.DataFrame([[matchweek,match_date, referee, stadium, attendance]],columns=['matchweek','match_date','referee','stadium','attendance'])
match_details_df['referee'] = match_details_df['referee'].str.replace('\n','').str.strip()
match_details_df['stadium'] = match_details_df['stadium'].str.replace('\n','').str.strip()
match_details_df['attendance'] = match_details_df['attendance'].str.replace('\n','').str.strip()
match_details_df['match_date'] = pd.to_datetime(match_details_df['match_date'],unit='ms')
for timeline_item in timeline_div.find_all('div',{'class':['home','away']}):
    player = timeline_item.find('a',{'class':'name'}).text
    player = player.split('\n')[0]
    
    event_df = pd.DataFrame([[timeline_item.find('a',{'class':'team'}).text,player,timeline_item.find('span',{'class':'visuallyHidden'}).text,timeline_item.find('time',{'class':'min'})]],columns=['team','player','event','minute'])
    match_df = match_df.append(event_df)
    
    if timeline_item.find('div',{'class':'subOn'}):
        sub_on = timeline_item.find('div',{'class':'subOn'}).find('a',{'class':'name'}).text
        sub_on = sub_on.split('\n')[0]
        event_df = pd.DataFrame([[timeline_item.find('a',{'class':'team'}).text,sub_on,'Substitution (on)',timeline_item.find('time',{'class':'min'})]],columns=['team','player','event','minute'])
        match_df = match_df.append(event_df)
match_df['team'] = match_df['team'].str.replace('\n','').str.strip()
print(match_details_df)
match_df

      matchweek          match_date    referee             stadium  \
0  Matchweek 38 2019-05-12 14:00:00  Mike Dean  Turf Moor, Burnley   

    attendance  
0  Att: 21,461  


,team,player,event,minute
0,BUR,1. Tom Heaton,Yellow Card,[3']
0,BUR,10. Ashley Barnes,Yellow Card,[22']
0,ARS,27. Konstantinos Mavropanos,Substitution,[34']
0,ARS,6. Laurent Koscielny,Substitution (on),[34']
0,BUR,5. James Tarkowski,Yellow Card,[45 +3']
0,BUR,2. Matthew Lowton,Yellow Card,[48']
0,ARS,14. Pierre-Emerick Aubameyang,Goal,[52']
0,ARS,29. Mattéo Guendouzi,Yellow Card,[52']
0,ARS,59. Joseph Willock,Substitution,[62']
0,ARS,49. Eddie Nketiah,Substitution (on),[62']


In [81]:
line_up_div = match_soup.find_all('div',{'class':'matchLineupTeamContainer'})
lineup_df = pd.DataFrame([],columns=['team','player','number','position','captain','substitution'])

for team in line_up_div:
    if team['data-index-class'] == 'home':
        home_away = team['data-index-class']
    else:
        home_away = 'away'
    for index, ul in enumerate(team.find_all('ul',{'class':'startingLineUpContainer'})):
        for player in ul.find_all('li',{'class':'player'}):
            player_lineup_df = pd.DataFrame([[home_away,
                                              player.find('div',{'class':'name'}).contents[0].strip(),
                                              player.find('div',{'class':'number'}).text.strip(),
                                              player.find('span',{'class':'position'}).text.strip(),
                                            index
                                             ]],columns=['team','player','number','position','substitution'])
            if player.find('div',{'class':'cpt'}):
                player_lineup_df['captain'] = 1
                player_lineup_df['position'] = player_lineup_df['position'].str.replace('C\n','')
            else:
                player_lineup_df['captain'] = 0
            lineup_df = lineup_df.append(player_lineup_df)
lineup_df

,team,player,number,position,captain,substitution
0,home,Tom Heaton,1,Goalkeeper,1,0
0,home,Charlie Taylor,3,Defender,0,0
0,home,Ben Mee,6,Defender,0,0
0,home,Matthew Lowton,2,Defender,0,0
0,home,James Tarkowski,5,Defender,0,0
0,home,Ashley Westwood,18,Midfielder,0,0
0,home,Jeff Hendrick,13,Midfielder,0,0
0,home,Dwight McNeil,31,Midfielder,0,0
0,home,Jack Cork,4,Midfielder,0,0
0,home,Ashley Barnes,10,Forward,0,0
